<a href="https://colab.research.google.com/github/KolliNagabhushanam/data-analysis/blob/main/Brain_tumor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
IMG_SIZE = 224
BATCH_SIZE = 32

In [2]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2)
train_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/train',
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training')

val_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/train',
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation')

test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    '/content/drive/MyDrive/text',
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary')

Found 2400 images belonging to 2 classes.
Found 600 images belonging to 2 classes.
Found 60 images belonging to 1 classes.


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
model = keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
history = model.fit(train_generator,validation_data=val_generator,epochs=5)

Epoch 1/5
75/75 [==============================] - 274s 4s/step - loss: 0.0286 - accuracy: 0.9904 - val_loss: 0.0155 - val_accuracy: 0.9917
Epoch 2/5
75/75 [==============================] - 266s 4s/step - loss: 0.0105 - accuracy: 0.9958 - val_loss: 0.0273 - val_accuracy: 0.9917
Epoch 3/5
75/75 [==============================] - 266s 4s/step - loss: 0.0205 - accuracy: 0.9912 - val_loss: 0.0399 - val_accuracy: 0.9900
Epoch 4/5
75/75 [==============================] - 268s 4s/step - loss: 0.0032 - accuracy: 0.9992 - val_loss: 0.0474 - val_accuracy: 0.9917
Epoch 5/5
75/75 [==============================] - 265s 4s/step - loss: 0.0081 - accuracy: 0.9975 - val_loss: 0.0191 - val_accuracy: 0.9967


In [6]:
model.save("model.h5","label.txt")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [25]:
from keras.models import load_model  # TensorFlow is required for Keras to work
from PIL import Image, ImageOps  # Install pillow instead of PIL
import numpy as np
from tensorflow.keras.preprocessing import image
model = load_model('/content/model.h5')
test_image_path = ('/content/drive/MyDrive/text/pred/pred0.jpg')
img =image.load_img(test_image_path, target_size=(224,224))
img_array=image.img_to_array(img)
img_array=np.expand_dims(img_array,axis=0)
img_array /=255
#classes
class_names = ['You have Brain Tumor','You do not have Brain Tumor']
data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)


In [28]:
image = Image.open("/content/drive/MyDrive/text/pred/pred0.jpg").convert("RGB")

# resizing the image to be at least 224x224 and then cropping from the center
size = (224, 224)
image = ImageOps.fit(image, size, Image.Resampling.LANCZOS)

# turn the image into a numpy array
image_array = np.asarray(image)

# Normalize the image
normalized_image_array = (image_array.astype(np.float32) / 127.5) - 1

# Load the image into the array
data[0] = normalized_image_array


In [29]:
#make prediction
prediction=model.predict(img_array)
# Predicts the model
prediction = model.predict(data)
index = np.argmax(prediction)
confidence_score = prediction[0][index]
# Print prediction and confidence score
if prediction<0.5:
  print("prediction : no tumer(probability:",prediction[0][0],")")
else:
  print("prediction : tumer present(probability:",prediction[0][0],")")
print("Result: ", class_names[index], end="")
print("\n")
print("Accuracy: ", confidence_score)

1/1 [==============================] - 0s 48ms/step
prediction : tumer present(probability: 0.5168407 )
Result:  You have Brain Tumor

Accuracy:  0.5168407
